## Importação das Bibliotecas

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dotenv import load_dotenv
import os

## Carregamento das Variáveis do .env

In [2]:
# Carregando as variáveis do arquivo .env
load_dotenv()

True

## API

In [3]:
# Configurações da API
URL_ARTILHEIROS = "https://v3.football.api-sports.io/players/topscorers"
API_KEY = os.getenv("API_KEY")

In [4]:
# Parâmetros para a requisição
PARAMS = {
    "season": "2022",  # Temporada de interesse
    "league": "71"     # 71 é o ID do Brasileirão
}

# Cabeçalhos para autenticação
HEADERS = {
    "x-rapidapi-host": "v3.football.api-sports.io",
    "x-rapidapi-key": API_KEY
}

In [5]:
# Requisição à API
response = requests.get(URL_ARTILHEIROS, headers=HEADERS, params=PARAMS)

In [6]:
# Verificação do status da resposta
if response.status_code == 200:
    # Parseando o JSON retornado
    data = response.json()
    
    # Verificando o campo "response"
    df_artilheiros = data.get("response", [])
    
    if df_artilheiros:
        print(f"Artilheiros encontrados: {len(df_artilheiros)}\n")
        
        # Criando uma lista para armazenar os dados
        players_data = []
        
        # Extraindo informações de cada jogador
        for player in df_artilheiros: 
            player_info = player.get("player", {})
            statistics = player.get("statistics", [{}])[0]
            team = statistics.get("team", {})
            goals = statistics.get("goals", {})
            
            # Adicionando os dados a uma lista como dicionário
            players_data.append({
                "Nome": player_info.get("name", "N/A"),
                "Time": team.get("name", "N/A"),
                "Gols": goals.get("total", 0),
                "Idade": player_info.get("age", "N/A"),
                "Nacionalidade": player_info.get("nationality", "N/A"),
                "Partidas": statistics.get("games", {}).get("appearences", 0),
                "Minutos Jogados": statistics.get("games", {}).get("minutes", 0),
                "Posição": player_info.get("position", "N/A"),
            })
        
        # Criando o DataFrame
        df_artilheiros = pd.DataFrame(players_data)
        
        # Exibindo as primeiras linhas do DataFrame
        print(df_artilheiros.head(31))
    else:
        print("Nenhum artilheiro encontrado.")
else:
    print(f"Erro na requisição: {response.status_code}")
    print("Detalhes:", response.text)

Artilheiros encontrados: 20

               Nome                 Time  Gols  Idade Nacionalidade  Partidas  \
0           G. Cano           Fluminense    26     36     Argentina        38   
1        Pedro Raul                Goias    19     28        Brazil        34   
2        J. Calleri            Sao Paulo    18     31     Argentina        36   
3           Bissoli                 Avai    14     26        Brazil        32   
4   Marcos Leonardo               Santos    13     21        Brazil        35   
5              Rony            Palmeiras    12     29        Brazil        33   
6              Hulk          Atletico-MG    12     38        Brazil        25   
7         D. Terans  Atletico Paranaense    12     30       Uruguay        31   
8             Pedro             Flamengo    11     27        Brazil        24   
9   Gabriel Barbosa             Flamengo    11     28        Brazil        29   
10          Luciano            Sao Paulo    11     31        Brazil        31   

## Criação do DataFrame pela Quantidade de Gols

In [7]:
# Resetando o índice para começar com 1
df_artilheiros.reset_index(drop=True, inplace=True)

# Ajustando o índice para começar com 1
df_artilheiros.index += 1


df_artilheiros

,Nome,Time,Gols,Idade,Nacionalidade,Partidas,Minutos Jogados,Posição
1,G. Cano,Fluminense,26,36,Argentina,38,3103,N/A
2,Pedro Raul,Goias,19,28,Brazil,34,2759,N/A
3,J. Calleri,Sao Paulo,18,31,Argentina,36,2821,N/A
4,Bissoli,Avai,14,26,Brazil,32,2460,N/A
5,Marcos Leonardo,Santos,13,21,Brazil,35,2893,N/A
6,Rony,Palmeiras,12,29,Brazil,33,2409,N/A
7,Hulk,Atletico-MG,12,38,Brazil,25,2096,N/A
8,D. Terans,Atletico Paranaense,12,30,Uruguay,31,2026,N/A
9,Pedro,Flamengo,11,27,Brazil,24,1167,N/A
10,Gabriel Barbosa,Flamengo,11,28,Brazil,29,2053,N/A


## Criação do DataFrame pelo Tempo Médio para Marcar

In [8]:
# Adicionando a coluna "Tempo por Gol" ao DataFrame
df_artilheiros['Tempo por Gol'] = df_artilheiros.apply(
    lambda row: row['Minutos Jogados'] / row['Gols'] if row['Gols'] > 0 else None, axis=1
)

# Adicionando a coluna "Média de Gols por Partida"
df_artilheiros['Média de Gols por Partida'] = df_artilheiros.apply(
    lambda row: row['Gols'] / row['Partidas'] if row['Partidas'] > 0 else None, axis=1
)

In [9]:
# Ordenando o DataFrame pela coluna 'Tempo por Gol' em ordem crescente
df_artilheiros_tempo_gol = df_artilheiros.sort_values(by='Tempo por Gol', ascending=True)

# Reiniciando o índice e descartando o índice antigo
df_artilheiros_tempo_gol.reset_index(drop=True, inplace=True)

# Ajustando o índice para começar com 1
df_artilheiros_tempo_gol.index += 1

# Exibindo o DataFrame com o índice reiniciado
df_artilheiros_tempo_gol


,Nome,Time,Gols,Idade,Nacionalidade,Partidas,Minutos Jogados,Posição,Tempo por Gol,Média de Gols por Partida
1,Pedro,Flamengo,11,27,Brazil,24,1167,N/A,106.090909,0.458333
2,G. Cano,Fluminense,26,36,Argentina,38,3103,N/A,119.346154,0.684211
3,Pedro Raul,Goias,19,28,Brazil,34,2759,N/A,145.210526,0.558824
4,J. Calleri,Sao Paulo,18,31,Argentina,36,2821,N/A,156.722222,0.500000
5,D. Terans,Atletico Paranaense,12,30,Uruguay,31,2026,N/A,168.833333,0.387097
6,Hulk,Atletico-MG,12,38,Brazil,25,2096,N/A,174.666667,0.480000
7,Bissoli,Avai,14,26,Brazil,32,2460,N/A,175.714286,0.437500
8,Yuri Alberto,Corinthians,8,23,Brazil,20,1451,N/A,181.375000,0.400000
9,Gabriel Barbosa,Flamengo,11,28,Brazil,29,2053,N/A,186.636364,0.379310
10,Luciano,Sao Paulo,11,31,Brazil,31,2055,N/A,186.818182,0.354839


## Criação do DataFrame pela Média de Gols por Partida

In [10]:
# Ordenando o DataFrame pela coluna 'Média de Gols por Partida' em ordem crescente
df_artilheiros_media_gol = df_artilheiros.sort_values(by='Média de Gols por Partida', ascending=False)

# Reiniciando o índice e descartando o índice antigo
df_artilheiros_media_gol.reset_index(drop=True, inplace=True)

# Ajustando o índice para começar com 1
df_artilheiros_media_gol.index += 1

# Exibindo o DataFrame com o índice reiniciado
df_artilheiros_media_gol

,Nome,Time,Gols,Idade,Nacionalidade,Partidas,Minutos Jogados,Posição,Tempo por Gol,Média de Gols por Partida
1,G. Cano,Fluminense,26,36,Argentina,38,3103,N/A,119.346154,0.684211
2,Pedro Raul,Goias,19,28,Brazil,34,2759,N/A,145.210526,0.558824
3,J. Calleri,Sao Paulo,18,31,Argentina,36,2821,N/A,156.722222,0.500000
4,Hulk,Atletico-MG,12,38,Brazil,25,2096,N/A,174.666667,0.480000
5,Pedro,Flamengo,11,27,Brazil,24,1167,N/A,106.090909,0.458333
6,Bissoli,Avai,14,26,Brazil,32,2460,N/A,175.714286,0.437500
7,Yuri Alberto,Corinthians,8,23,Brazil,20,1451,N/A,181.375000,0.400000
8,D. Terans,Atletico Paranaense,12,30,Uruguay,31,2026,N/A,168.833333,0.387097
9,Gabriel Barbosa,Flamengo,11,28,Brazil,29,2053,N/A,186.636364,0.379310
10,Marcos Leonardo,Santos,13,21,Brazil,35,2893,N/A,222.538462,0.371429


In [11]:
# Exportando o DataFrame para um arquivo Excel
df_artilheiros.to_excel('artilheiros_brasileirao_2022.xlsx', index=False, engine='openpyxl')

# Mensagem de confirmação
print("DataFrame exportado com sucesso para 'artilheiros.xlsx'")

DataFrame exportado com sucesso para 'artilheiros.xlsx'
